문제 1
=========

In [144]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.stats.anova as anova
import scipy as sp
import sklearn.decomposition as sd

In [145]:
D=pd.read_excel('https://data.hossam.kr/E04/diabetes.xlsx')
data=pd.DataFrame(D)

In [146]:
y=data["target"]
x=data[data.columns.difference(["target"])]

VIF

In [147]:
import statsmodels.stats.outliers_influence as ssoi

In [148]:
ssoi.variance_inflation_factor

<function statsmodels.stats.outliers_influence.variance_inflation_factor(exog, exog_idx)>

In [149]:
VIF=pd.DataFrame()
VIF["vif_value"]=[ssoi.variance_inflation_factor(x.values,i) for i in range(x.shape[1])]
VIF["name"]=x.columns
VIF.set_index("name",inplace=True)

In [150]:
# while not all(VIF["vif_value"]<=10):
for i in VIF.index:
    if VIF.loc[i,"vif_value"]==max(VIF["vif_value"]):
        VIF.drop(i,axis=0,inplace=True)

In [151]:
VIF

,vif_value
name,
age,1.217307
bmi,1.509437
bp,1.459428
s4,8.890986
s6,1.484623
sex,1.278071


In [152]:
VIF.index

Index(['age', 'bmi', 'bp', 's4', 's6', 'sex'], dtype='object', name='name')

OLS 우선 실시

In [153]:
cols="+".join(VIF.index)
formula=f"target~{cols}"

In [154]:
model=smf.ols(formula,data)
model.fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.449
Model:                            OLS   Adj. R-squared:                  0.441
Method:                 Least Squares   F-statistic:                     59.01
Date:                Wed, 26 Jul 2023   Prob (F-statistic):           2.73e-53
Time:                        16:33:31   Log-Likelihood:                -2415.6
No. Observations:                 442   AIC:                             4845.
Df Residuals:                     435   BIC:                             4874.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    152.1335      2.741     55.496      0.000     146.746     157.521
age          -13.3215     62.658     -0.213      0.832    -136.472     109.829
bmi          619.0558     68.629      9.020      0.000     484.170     753.941
bp           384.6380     68.183      5.641      0.000     250.628     518.648
s4           365.6936     69.494      5.262      0.000     229.107     502.280
s6           126.2927     68.994      1.830      0.068      -9.310     261.895
sex         -222.9869     62.680     -3.558      0.000    -346.180     -99.794
==============================================================================
Omnibus:                        6.296   Durbin-Watson:                   1.875
Prob(Omnibus):                  0.043   Jarque-Bera (JB):                4.421
Skew:                           0.099   Prob(JB):                        0.110
Kurtosis:                       2.552   Cond. No.                         31.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

문제 2
==============

In [155]:
Data2=pd.read_excel('https://data.hossam.kr/E04/manhattan.xlsx')
data2=pd.DataFrame(Data2)

In [156]:
data2.drop('borough',axis=1,inplace=True)

In [157]:
data2['neighborhood'].nunique()

32

In [158]:
y2=data2['rent']
x2=data2[data2.columns.difference(['rent',"neighborhood"])]

VIF

In [159]:
VIF=pd.DataFrame()
VIF["vif_value"]=[ssoi.variance_inflation_factor(x2.values,i) for i in range(x2.shape[1])]
VIF.index=x2.columns

In [160]:
VIF.T

,bathrooms,bedrooms,building_age_yrs,floor,has_dishwasher,has_doorman,has_elevator,has_gym,has_patio,has_roofdeck,has_washer_dryer,min_to_subway,no_fee,size_sqft
vif_value,16.97275,7.804207,2.162425,2.324919,1.730935,3.286035,3.5862,2.572165,1.10204,1.893996,1.626896,1.778081,1.631876,18.068311


In [161]:
while not all(VIF['vif_value']<=10):
    for i in VIF.index:
        if VIF.loc[i,"vif_value"]==max(VIF['vif_value']):
            VIF.drop(i,axis=0,inplace=True)
            break

In [186]:
VIF

,vif_value
bedrooms,7.804207
building_age_yrs,2.162425
floor,2.324919
has_dishwasher,1.730935
has_doorman,3.286035
has_elevator,3.586200
has_gym,2.572165
has_patio,1.102040
has_roofdeck,1.893996
has_washer_dryer,1.626896


In [162]:
idx=list(VIF.index)
idx.append("neighborhood")

In [163]:
pre_x2=data2.drop("rent",axis=1)[idx]

In [172]:
pre_x2

,bedrooms,building_age_yrs,floor,has_dishwasher,has_doorman,has_elevator,has_gym,has_patio,has_roofdeck,has_washer_dryer,min_to_subway,no_fee,neighborhood
0,0.0,17,2.0,1,0,1,1,0,1,0,9,1,Upper East Side
1,2.0,96,1.0,0,0,0,0,0,0,0,4,0,Greenwich Village
2,1.0,29,51.0,1,1,1,0,0,1,0,2,0,Midtown
3,1.0,31,8.0,1,1,1,1,0,0,0,3,0,Greenwich Village
4,2.0,136,4.0,1,1,1,1,0,0,0,3,0,Soho
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,1.0,16,8.0,1,1,1,1,0,1,1,3,1,Chelsea
3535,2.0,9,10.0,1,1,1,1,0,1,1,5,1,Tribeca
3536,0.0,96,5.0,0,0,0,0,0,0,0,2,0,Little Italy
3537,1.0,14,5.0,1,1,1,1,0,0,1,6,1,Midtown West


In [165]:
import statsmodels.api as sm

In [173]:
use_x2=pd.get_dummies(pre_x2,columns=['neighborhood'],drop_first=True)

In [180]:
use_x2.rename(columns=dict(zip(use_x2.columns,map(lambda x:x.replace(" ","_"),use_x2.columns))),inplace=True)
use_x2.rename(columns=dict(zip(use_x2.columns,map(lambda x:x.replace("/","_"),use_x2.columns))),inplace=True)

OLS

In [181]:
add="+".join(use_x2.columns)

In [182]:
formula=f'rent~{add}'

In [183]:
formula

'rent~bedrooms+building_age_yrs+floor+has_dishwasher+has_doorman+has_elevator+has_gym+has_patio+has_roofdeck+has_washer_dryer+min_to_subway+no_fee+neighborhood_Central_Harlem+neighborhood_Central_Park_South+neighborhood_Chelsea+neighborhood_Chinatown+neighborhood_East_Harlem+neighborhood_East_Village+neighborhood_Financial_District+neighborhood_Flatiron+neighborhood_Gramercy_Park+neighborhood_Greenwich_Village+neighborhood_Hamilton_Heights+neighborhood_Inwood+neighborhood_Little_Italy+neighborhood_Long_Island_City+neighborhood_Lower_East_Side+neighborhood_Manhattanville+neighborhood_Midtown+neighborhood_Midtown_East+neighborhood_Midtown_South+neighborhood_Midtown_West+neighborhood_Morningside_Heights+neighborhood_Nolita+neighborhood_Roosevelt_Island+neighborhood_Soho+neighborhood_Stuyvesant_Town_PCV+neighborhood_Tribeca+neighborhood_Upper_East_Side+neighborhood_Upper_West_Side+neighborhood_Washington_Heights+neighborhood_West_Harlem+neighborhood_West_Village'

In [184]:
ols=smf.ols(formula,pd.merge(y2,use_x2,left_index=True,right_index=True))

In [185]:
ols.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   rent   R-squared:                       0.574
Model:                            OLS   Adj. R-squared:                  0.569
Method:                 Least Squares   F-statistic:                     109.5
Date:                Wed, 26 Jul 2023   Prob (F-statistic):               0.00
Time:                        16:40:11   Log-Likelihood:                -32033.
No. Observations:                3539   AIC:                         6.415e+04
Df Residuals:                    3495   BIC:                         6.442e+04
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                 2561.4087    240.494     10.651      0.000    2089.886    3032.931
neighborhood_Central_Harlem[T.True]      -2532.7576    315.779     -8.021      0.000   -3151.887   -1913.628
neighborhood_Central_Park_South[T.True]   1858.4650    486.505      3.820      0.000     904.603    2812.327
neighborhood_Chelsea[T.True]               992.3544    261.969      3.788      0.000     478.727    1505.982
neighborhood_Chinatown[T.True]           -1847.1339    766.799     -2.409      0.016   -3350.552    -343.715
neighborhood_East_Harlem[T.True]         -2843.0076    387.973     -7.328      0.000   -3603.685   -2082.330
neighborhood_East_Village[T.True]         -270.6996    293.473     -0.922      0.356    -846.095     304.696
neighborhood_Financial_District[T.True]   -295.3723    254.134     -1.162      0.245    -793.639     202.894
neighborhood_Flatiron[T.True]              696.2149    281.621      2.472      0.013     144.056    1248.374
neighborhood_Gramercy_Park[T.True]         160.8651    339.482      0.474      0.636    -504.737     826.467
neighborhood_Greenwich_Village[T.True]    1412.4107    338.687      4.170      0.000     748.367    2076.455
neighborhood_Hamilton_Heights[T.True]    -2444.9261    564.511     -4.331      0.000   -3551.730   -1338.122
neighborhood_Inwood[T.True]              -2955.2061    638.561     -4.628      0.000   -4207.196   -1703.217
neighborhood_Little_Italy[T.True]         -418.5174   1222.165     -0.342      0.732   -2814.748    1977.713
neighborhood_Long_Island_City[T.True]    -1432.7430   1061.838     -1.349      0.177   -3514.629     649.143
neighborhood_Lower_East_Side[T.True]      -634.5628    388.619     -1.633      0.103   -1396.506     127.381
neighborhood_Manhattanville[T.True]      -2606.8072   2089.600     -1.248      0.212   -6703.766    1490.152
neighborhood_Midtown[T.True]                48.0448    291.767      0.165      0.869    -524.007     620.097
neighborhood_Midtown_East[T.True]         -787.9160    229.682     -3.430      0.001   -1238.241    -337.591
neighborhood_Midtown_South[T.True]        -685.0653    313.025     -2.189      0.029   -1298.795     -71.336
neighborhood_Midtown_West[T.True]         -985.9022    240.135     -4.106      0.000   -1456.721    -515.084
neighborhood_Morningside_Heights[T.True] -2391.6812    619.351     -3.862      0.000   -3606.007   -1177.355
neighborhood_Nolita[T.True]                308.5021    725.872      0.425      0.671   -1114.674    1731.678
neighborhood_Roosevelt_Island[T.True]    -1979.3005    953.321     -2.076      0.038   -3848.423    -110.178
neighborhood_Soho[T.True]                 2702.8136    351.730      7.684      0.000    2013.196    3392.431
neighborhood_Stuyvesant_Town_PC